In [13]:
file_path = '../improve_data/enhance-rare-train-val-all.json'

In [18]:
import json
with open(file_path, 'r', encoding='utf-8') as file:
    input_data = json.load(file)
count = 0
train_re_types = ["时间","相关主体","类型","组织机构","地点","军种", "单位", "任务状态"]

train_re_counts = [849, 1383, 1729, 1143, 879, 952, 504, 952]

test_re_counts = [98, 147, 184, 123, 92, 120, 120, 120]  

test_re_F1 = [0.857, 0.537, 0.516, 0.593, 0.826, 0, 0, 1]

for item in input_data:
    for triple in item["relation"]:
        if triple["relation"] == "时间":
            count+=1

print(count)

for i in range(len(train_re_types)):
    condition = {
        "关系": train_re_types[i],
        "训练集中数量": train_re_counts[i],
        "测试集中数量": test_re_counts[i],
        "F1值": test_re_F1[i]
    }
    print(condition)

849
{'关系': '时间', '训练集中数量': 849, '测试集中数量': 98, 'F1值': 0.857}
{'关系': '相关主体', '训练集中数量': 1383, '测试集中数量': 147, 'F1值': 0.537}
{'关系': '类型', '训练集中数量': 1729, '测试集中数量': 184, 'F1值': 0.516}
{'关系': '组织机构', '训练集中数量': 1143, '测试集中数量': 123, 'F1值': 0.593}
{'关系': '地点', '训练集中数量': 879, '测试集中数量': 92, 'F1值': 0.826}
{'关系': '军种', '训练集中数量': 952, '测试集中数量': 120, 'F1值': 0}
{'关系': '单位', '训练集中数量': 504, '测试集中数量': 120, 'F1值': 0}
{'关系': '任务状态', '训练集中数量': 952, '测试集中数量': 120, 'F1值': 1}


- 主体名称、指挥员训练和测试数量为0：前者是构建数据集的一个if语句逻辑不严谨，已修正；后者是原始数据集中指挥员没有有效数据，全是未知
- 从数据上来看，训练集中关系数量和最后该关系的推理准确率没有明确的相关性（参考——时间、组织机构、地点）
- 【时间】相比来说F1更低的原因：
  - 情况1：文本中本身不包含时间，但表格中有时间信息（时间信息可能还不真实），而且存在过拟合问题，时间抽取被干扰，既不是训练集数据也不是测试集数据【数据2、5】
  - 情况2：表格中两行对应两个不同事件，但是同一个文本，一条做训练集了一条做测试集了，也是过拟合问题【数据3、7、9、10】
  - 情况3：直接就是没抽出来【数据8】
- 【地点、组织机构】F1值比较可观的原因：同上，很多数据过拟合时地点和组织机构是训练数据和测试数据相同的
- 【主体名称、主体类型、主体数量】：
  - 存在一个表格内多个内容的情况，需要新加拆分规则。但看这一条抽取效果还行【数据1】
  - 抽取结果也不能说不对，反而可能是更贴近正常理解，但和原始数据对不上【数据3】
  - 两个训练集数据中，都只有主体名称，结果抽取时过拟合+大模型理解，把两个主体名称一个抽成主体名称一个抽成类型【数据4】
  - 过拟合问题也存在

总结：
1. 过拟合问题：原始数据是两行表格，数量是7+8=15的情况，文本中出现的是15，一条7做训练集了，一条8在测试集，最后抽取出来就是数量这一栏不对，其他过拟合【数据2】。**既会导致一些尾实体F1值虚高，又会干扰一些尾实体的抽取**。
2. 合理怀疑目前抽取效果较好的情况有很多是由于过拟合
3. 头实体，即事件抽取也有存在偏差的情况（虽然计算F1时没有用到头实体的抽取情况）【数据6】，说明这种构建关系的方式大模型还是可以理解，不过关系的构建头尾实体可能还是用名词性实体更好
4. 还是原始的数据标注存在很多问题，数据质量高的话效果会好很多【数据6，唯一的错误就是头实体事件的抽取不一致】